In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [14]:
from datetime import datetime

date = '2023-11-01'
format_date = datetime.strptime(date, "%Y-%m-%d")

day = format_date.strftime("%d")
month = format_date.strftime("%m")
year = format_date.strftime("%Y")

print("Day:", day)
print("Month:", month)
print("Year:", year)


Day: 01
Month: 11
Year: 2023


In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

url = "https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal=01&bulan=11&tahun=2023&pos=indeks&per_page=60"

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Check if 'list-berita' div is found
body = soup.find('div', {"class": "list-berita"})
if body:
    # If found, proceed with finding 'li' elements
    articles = body.find_all('li')

    for article in articles:
        link = article.find('a')['href']
        print(link)
else:
    print("Couldn't find the 'list-berita' div.")


Couldn't find the 'list-berita' div.


In [29]:
def scrape_links(date, page_number):
    format_date = datetime.strptime(date, "%Y-%m-%d")
    day = format_date.strftime("%d")
    month = format_date.strftime("%m")
    year = format_date.strftime("%Y")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal={day}&bulan={month}&tahun={year}&pos=indeks&per_page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('div', {"class": "list-berita"})
    links = []  # Initialize links outside the if block

    if body:
        articles = body.find_all('li')
        for article in articles:
            link = article.find('a')['href']
            links.append(link)
    else:
        print("Couldn't find the 'list-berita' div.")

    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    return links


In [30]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 0
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 20

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [31]:
link=scrape_link_per_day('2023-11-01')

Scraped 20 links from page 0 url https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal=01&bulan=11&tahun=2023&pos=indeks&per_page=0
Scraped 20 links from page 20 url https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal=01&bulan=11&tahun=2023&pos=indeks&per_page=20
Scraped 8 links from page 40 url https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal=01&bulan=11&tahun=2023&pos=indeks&per_page=40
Couldn't find the 'list-berita' div.
Scraped 0 links from page 60 url https://www.kontan.co.id/search/indeks?kanal=nasional&tanggal=01&bulan=11&tahun=2023&pos=indeks&per_page=60


In [49]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"class": "detail-desk"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "fs14 ff-opensans font-gray"})
                    # print(date_elem)
                    if date_elem:
                        date_text= date_elem.text.strip()
                        month_mapping = {
                            'Januari': '01',
                            'Februari': '02',
                            'Maret': '03',
                            'April': '04',
                            'Mei': '05',
                            'Juni': '06',
                            'Juli': '07',
                            'Agustus': '08',
                            'September': '09',
                            'Oktober': '10',
                            'November': '11',
                            'Desember': '12'
                        }

                        # Memisahkan string tanggal dan waktu
                        date_time_parts = date_text.split('/')[0].strip().split()
                        day = date_time_parts[1]
                        month_name = date_time_parts[2]
                        year = date_time_parts[3]

                        # Mengonversi nama bulan menjadi angka menggunakan kamus
                        month = month_mapping[month_name]

                        # Membentuk string tanggal yang diformat
                        formatted_date = f'{year}-{month}-{day}'

                        # date_text = date_text
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "tmpt-desk-kon"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [50]:
url='https://nasional.kontan.co.id/news/ini-data-dan-informasi-yang-harus-disetorkan-pmse-ke-bps'
data_kontan = scrape_url(url)

In [51]:
data_kontan

{'title': 'Ini Data dan Informasi yang Harus Disetorkan PMSE ke BPS',
 'date': '2023-11-01',
 'content': 'Reporter: Bidara Pink | Editor: Khomarul HidayatKONTAN.CO.ID - JAKARTA. Makin berkembangnya transaksi dan ekonomi digital, Badan Pusat Statistik (BPS) pun meluncurkan aturan baru terkait dengan Perdagangan Melalui Sistem Elektronik (PMSE).Ini tertuang dalam Peraturan BPS Nomor 4 Tahun 2023 tentang Penyampaian dan Pengelolaan Data dan/atau Informasi PMSE.Dengan adanya beleid tersebut, maka pelaku PMSE wajib lapor kepada BPS mengenai sejumlah data atau informasi dari perusahaannya secara berkala.Dalam Pasal 4 ayat (1) tertulis, data yang wajib disetor kepada otoritas statistik adalah keterangan umum perusahaan, tenaga kerja, pendapatan dan pengeluaran.Pelaku PMSE juga wajib setor informasi kategori produk, kategori wilayah, transaksi, metode pembayaran, juga jumlah penjual dan pembeli.Dalam Pasal 6 ayat (1) kemudian dijelaskan, bahwa para pelaku PMSE bisa menggunakan moda penyampaian